In [1]:
import os

import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from pystan import StanModel
from sklearn.preprocessing import scale

os.environ['PYSTAN_CACHE_PATH'] = '/home/gianluca/git/rethinking/cache'
os.environ['PYSTAN_MODEL_PATH'] = '/home/gianluca/git/rethinking/models'

from rethinking import StanCache
from rethinking.plotting import summaryplot

output_notebook()

Loading BokehJS ...

In [2]:
DATA_FOLDER = '../data'
DIVORCE_FULLPATH = os.path.join(DATA_FOLDER, 'WaffleDivorce.csv')

In [3]:
divorce = pd.read_csv(DIVORCE_FULLPATH)
divorce.head()

,Location,Loc,Population,MedianAgeMarriage,Marriage,Marriage SE,Divorce,Divorce SE,WaffleHouses,South,Slaves1860,Population1860,PropSlaves1860
0,Alabama,AL,4.78,25.3,20.2,1.27,12.7,0.79,128,1,435080,964201,0.45
1,Alaska,AK,0.71,25.2,26.0,2.93,12.5,2.05,0,0,0,0,0.00
2,Arizona,AZ,6.33,25.8,20.3,0.98,10.8,0.74,18,0,0,0,0.00
3,Arkansas,AR,2.92,24.3,26.4,1.70,13.5,1.22,41,1,111115,435450,0.26
4,California,CA,37.25,26.8,19.1,0.39,8.0,0.24,0,0,0,379994,0.00


$$
\begin{align*}
    D_i &\sim Normal(\mu_i, \sigma) \\
    \mu_i &= \alpha + \beta_A A_i \\
    \alpha &\sim Normal(10, 10) \\
    \beta_A &\sim Normal(0, 1) \\
    \sigma &\sim Uniform(0, 10)
\end{align*}
$$

In [4]:
m5_1 = StanCache(model_name='m5_1').compile()
m5_1.model_code

data {
    int<lower=0> N;
    vector[N] X;
    vector<lower=0, upper=100>[N] divorce_rate;
}
parameters {
    real alpha;
    real beta;
    real<lower=0> sigma;
}
transformed parameters {
    vector<lower=-100, upper=100>[N] mu;

    mu = alpha + X * beta;
}
model {
    alpha ~ normal(10, 10);
    beta ~ normal(0, 1);
    sigma ~ uniform(0, 10);
    divorce_rate ~ normal(mu, sigma);
}



In [5]:
divorce['MedianAgeMarriage_s'] = scale(
    X=divorce.loc[:, ['MedianAgeMarriage']], with_mean=True, with_std=True)
divorce['Divorce_pct'] = divorce.loc[:, 'Divorce'] / 100

In [6]:
divorce_data = dict(
    N=divorce.shape[0],
    K=1,
    X=divorce.MedianAgeMarriage_s,
    divorce_rate=divorce.Divorce
)

fit5_1 = m5_1.sampling(data=divorce_data, iter=500)
print(fit5_1.stansummary(pars=['alpha', 'beta', 'sigma']))

Inference for Stan model: anon_model_069dd64a1ee6b8cdacde859166ae13ef.
4 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=1000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha   9.69  6.7e-3   0.21   9.27   9.55   9.69   9.82  10.11   1000    1.0
beta   -1.02  8.7e-3    0.2   -1.4  -1.16  -1.03  -0.89  -0.64    521    1.0
sigma   1.51  4.9e-3   0.15   1.24    1.4    1.5   1.61   1.83    977    1.0

Samples were drawn using NUTS at Mon Mar  5 20:17:38 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


/home/gianluca/.virtualenvs/rethinking/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [7]:
summaryplot(fit5_1, pars=['alpha', 'beta', 'sigma'])

In [9]:
divorce['Marriage_s'] = scale(
    X=divorce.loc[:, ['Marriage']], with_mean=True, with_std=True)

In [10]:
divorce2_data = dict(
    N=divorce.shape[0],
    K=1,
    X=divorce.Marriage_s,
    divorce_rate=divorce.Divorce
)

fit5_2 = m5_1.sampling(data=divorce2_data, iter=500)
print(fit5_2.stansummary(pars=['alpha', 'beta', 'sigma']))

/home/gianluca/.virtualenvs/rethinking/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


Inference for Stan model: anon_model_069dd64a1ee6b8cdacde859166ae13ef.
4 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=1000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha   9.68  7.9e-3   0.24   9.19   9.52   9.67   9.83  10.16    945    1.0
beta    0.64  9.6e-3   0.24   0.15    0.5   0.65   0.81   1.12    620    1.0
sigma   1.75  7.0e-3    0.2   1.42   1.61   1.73   1.87    2.2    816    1.0

Samples were drawn using NUTS at Mon Mar  5 20:18:28 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


/home/gianluca/git/rethinking/src/rethinking/stan.py:97: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(results, f)


# Multivariate linear model


$$
\begin{align}
    D_i &\sim Normal(\mu_i, \sigma) \\
    \mu_i &= \alpha + \sum_{j=1}^{n} \beta_i x_{ji} \\
    \alpha &\sim Normal(10, 10) \\
    \beta_R &\sim Normal(0, 1) \\
    \beta_A &\sim Normal(0, 1) \\
    \sigma &\sim Uniform(0, 10)
\end{align}
$$

In [11]:
m5_3 = StanCache(model_name='m5_3').compile()
m5_3.model_code

data {
    int<lower=0> N;
    int<lower=0> K;
    matrix[N, K] X;
    vector<lower=0, upper=100>[N] divorce_rate;
}
parameters {
    real alpha;
    vector[K] beta;
    real<lower=0> sigma;
}
transformed parameters {
    vector<lower=-100, upper=100>[N] mu;

    mu = alpha + X * beta;
}
model {
    alpha ~ normal(10, 10);
    beta ~ normal(0, 1);
    sigma ~ uniform(0, 10);
    divorce_rate ~ normal(mu, sigma);
}



In [12]:
divorce_multi_data = dict(
    N=divorce.shape[0],
    K=2,
    X=divorce.loc[:, ['Marriage_s', 'MedianAgeMarriage_s']],
    divorce_rate = divorce.Divorce
)

fit5_3 = m5_3.sampling(data=divorce_multi_data, iter=500)
print(fit5_3.stansummary(pars=['alpha', 'beta', 'sigma']))

Inference for Stan model: anon_model_623c071ba8c404bae8e2868021df5aa8.
4 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=1000.

          mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha     9.69  7.5e-3   0.22   9.25   9.56   9.69   9.84  10.11    844    1.0
beta[0]  -0.14    0.02    0.3  -0.72  -0.34  -0.15   0.06   0.46    340   1.02
beta[1]  -1.13    0.02   0.31  -1.71  -1.34  -1.14  -0.93   -0.5    316   1.02
sigma     1.53  5.8e-3   0.16   1.25   1.42   1.51   1.62   1.88    811    1.0

Samples were drawn using NUTS at Sun Mar  4 12:44:15 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


/home/gianluca/.virtualenvs/rethinking/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [13]:
summaryplot(fit5_3, pars=['alpha', 'beta', 'sigma'])

# Predictor residual plots